# MMA 662 - Assignment 2





Installing Gurobi package:

In [116]:
# Uncomment if working with Colab:
# !pip install gurobipy 
import gurobipy as gp
from gurobipy import *

import numpy as np

## 1: The Apportionment Problem.

Dataset:


In [88]:
# Population list:
Pop = np.array([5300000, 9200000, 5400000, 8000000, 3200000, 
                8800000, 8100000, 10100000, 3600000, 6200000, 
                11700000, 5800000, 7000000, 6300000, 4800000, 
                10700000, 7000000, 9400000, 6400000, 9100000])

# Quota (q_i) list:
Quota = np.array([21.80219, 37.84531, 22.21355, 32.90897, 13.16359, 
                  36.19986, 33.32033, 41.54757, 14.80903, 25.50445, 
                  48.12936, 23.859, 28.79535, 25.91581, 19.74538, 
                  44.01574, 28.79535, 38.66804, 26.32717, 37.43395])

# Province list:
Province = np.array(['P0', 'P1', 'P2', 'P3', 'P4', 
                      'P5', 'P6', 'P7', 'P8', 'P9', 
                      'P10', 'P11', 'P12', 'P13', 'P14', 
                      'P15', 'P16', 'P17', 'P18', 'P19'])

# Total seats:
s = 601

### 1.1: Minimize the sum of absolute gaps.

In [3]:
# Your solution for 1.1

In [33]:
prob1 = gp.Model("Min Sum Abs gaps")

In [34]:
L = len(Province)

In [35]:
z = prob1.addVars(L, lb = 0, vtype = GRB.CONTINUOUS, name = ["Devation from fair # Seats for provice "+str(i) for i in range(L)])
a = prob1.addVars(L, lb = 0, vtype = GRB.INTEGER, name = ["Fair # Seats for provice "+str(i) for i in range(L)])

In [36]:
prob1.setObjective(sum(z[i] for i in range(L)), GRB.MINIMIZE)

In [37]:
for i in range(L):
    prob1.addConstr(z[i] >= Quota[i] - a[i])
    prob1.addConstr(z[i] >= a[i] - Quota[i])

In [38]:
prob1.addConstr(sum(a[i] for i in range(L)) == s)

<gurobi.Constr *Awaiting Model Update*>

In [39]:
prob1.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 41 rows, 40 columns and 100 nonzeros
Model fingerprint: 0x0ac73334
Variable types: 20 continuous, 20 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 6e+02]
Found heuristic solution: objective 1149.3456604
Presolve time: 0.00s
Presolved: 41 rows, 40 columns, 100 nonzeros
Variable types: 20 continuous, 20 integer (0 binary)

Root relaxation: objective 0.000000e+00, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   20 1149.34566    0.00000   100%     -    0s
H    0     0                       5.2114800    0.00000   100%     -    0s
H    0 

In [41]:
for v in prob1.getVars():
    print(v.varName, "=", round(v.x))

Devation from fair # Seats for provice 0 = 0
Devation from fair # Seats for provice 1 = 0
Devation from fair # Seats for provice 2 = 0
Devation from fair # Seats for provice 3 = 0
Devation from fair # Seats for provice 4 = 0
Devation from fair # Seats for provice 5 = 0
Devation from fair # Seats for provice 6 = 0
Devation from fair # Seats for provice 7 = 0
Devation from fair # Seats for provice 8 = 0
Devation from fair # Seats for provice 9 = 1
Devation from fair # Seats for provice 10 = 0
Devation from fair # Seats for provice 11 = 0
Devation from fair # Seats for provice 12 = 0
Devation from fair # Seats for provice 13 = 0
Devation from fair # Seats for provice 14 = 0
Devation from fair # Seats for provice 15 = 0
Devation from fair # Seats for provice 16 = 0
Devation from fair # Seats for provice 17 = 0
Devation from fair # Seats for provice 18 = 0
Devation from fair # Seats for provice 19 = 0
Fair # Seats for provice 0 = 22
Fair # Seats for provice 1 = 38
Fair # Seats for provice 2

In [ ]:
# This linearization is valid because it does not change the model. It holds all the model's considerations and do not add any additional constraints.
# Having the combination of z[i] >= Quota[i] - a[i] and z[i] >= -(Quota[i] - a[i]), the only possible value for z[i] is
# equal to abs(Quota[i] - a[i]). Also, all the constraints and the objective function is linear; so, the problem would become
# a linear optimization problem.

# The number of decision variables is twice the number of provinces; one for the difference between the allocated seats and
# the fair number of seats which may not be integer (z[i]), and one the final number of allocated seats to each province that has to be
# an integer (a[i]). Therefore, in this problem, we will have 2*20=40 decision variables.

# In this problem, we have a constraint for the total number of seats. 
# Also, for each province, we will have two constraints resulting from the z[i] = abs(Quota[i] - a[i]).
# Therefore, the total number of constraints is 2*20 + 1 = 41.

### 1.2: Min-max.

In [4]:
# Your solution for 1.2

In [42]:
prob2 = gp.Model("Min Max")

In [43]:
L = len(Province)

In [44]:
a = prob2.addVars(L, lb = 0, vtype = GRB.INTEGER, name = ["Fair # Seats for provice "+str(i) for i in range(L)])
z = prob2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "max difference between fair # seats and allocated # seats")

In [45]:
prob2.setObjective(z, GRB.MINIMIZE)

In [46]:
for i in range(L):
    prob2.addConstr(z >= Quota[i] - a[i])
    prob2.addConstr(z >= a[i] - Quota[i])

In [47]:
prob2.addConstr(sum(a[i] for i in range(L)) == s)

<gurobi.Constr *Awaiting Model Update*>

In [48]:
prob2.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 41 rows, 21 columns and 100 nonzeros
Model fingerprint: 0x5c9ced0b
Variable types: 1 continuous, 20 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 6e+02]
Found heuristic solution: objective 574.6728301
Presolve time: 0.00s
Presolved: 41 rows, 21 columns, 100 nonzeros
Variable types: 1 continuous, 20 integer (0 binary)

Root relaxation: objective 0.000000e+00, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   20  574.67283    0.00000   100%     -    0s
H    0     0                       0.5475700    0.00000   100%     -    0s
H    0    

In [49]:
for v in prob2.getVars():
    print(v.varName, "=", round(v.x))

Fair # Seats for provice 0 = 22
Fair # Seats for provice 1 = 38
Fair # Seats for provice 2 = 22
Fair # Seats for provice 3 = 33
Fair # Seats for provice 4 = 13
Fair # Seats for provice 5 = 36
Fair # Seats for provice 6 = 33
Fair # Seats for provice 7 = 42
Fair # Seats for provice 8 = 15
Fair # Seats for provice 9 = 25
Fair # Seats for provice 10 = 48
Fair # Seats for provice 11 = 24
Fair # Seats for provice 12 = 29
Fair # Seats for provice 13 = 26
Fair # Seats for provice 14 = 20
Fair # Seats for provice 15 = 44
Fair # Seats for provice 16 = 29
Fair # Seats for provice 17 = 39
Fair # Seats for provice 18 = 26
Fair # Seats for provice 19 = 37
max difference between fair # seats and allocated # seats = 1


In [31]:
# This linearization is valid since it keeps all the consideration of the model without adding or removing any points.
# Since we have z >= Quota[i] - a[i] for i in range(L), it would result in z >= max(Quota[i] - a[i]).
# Also, z <= a[i] - Quota[i] for i in range(L) constraints would result in z <= max(Quota[i] - a[i]).
# Therefore, these two sets of constraints would force z to be equal to z = max(Quota[i] - a[i] for i in range(L))

# In this formulation, we have a single decision variable z, which we try to minimize in addition to the fair number of
# seats for each province. Therefore, the model would have a total of 20+1=21 decision variables.

# Similar to the previous formulation, we will have two constraints for z per province. 
# We also have a constraint for the total number of seats.
# Therefore, the model would have 2*20 + 1 = 41 constraints.

### 1.3: Political considerations.

In [5]:
# Your solution for 1.3

In [ ]:
# Generally, the formulation for this problem would be similar to the problem 1.1. 
# We will have a new constraint to capture requirement (i) added to the problem (a[i] == a[j])
# , 3 new constraints to capture requirement (ii) (a[i] <= Quota[i]),
# and one new constraint to capture requirement (iii)

In [89]:
prob3 = gp.Model("Political Considerations")

In [90]:
L = len(Province)

In [91]:
z = prob3.addVars(L, lb = 0, vtype = GRB.CONTINUOUS, name = ["Devation from fair # Seats for provice "+str(i) for i in range(L)])
a = prob3.addVars(L, lb = 0, vtype = GRB.INTEGER, name = ["Fair # Seats for provice "+str(i) for i in range(L)])

In [92]:
prob3.setObjective(sum(z[i] for i in range(L)), GRB.MINIMIZE)

In [93]:
for i in range(L):
    prob3.addConstr(z[i] >= Quota[i] - a[i])
    prob3.addConstr(z[i] >= a[i] - Quota[i])

In [94]:
prob3.addConstr(sum(a[i] for i in range(L)) == s)

<gurobi.Constr *Awaiting Model Update*>

In [95]:
prob3.addConstr(a[4] == a[8])

<gurobi.Constr *Awaiting Model Update*>

In [96]:
for i in [7, 10, 15]:
    prob3.addConstr(a[i] <= Quota[i])

In [98]:
prob3.addConstr(a[2] - Quota[2] >= a[0] - Quota[0])

<gurobi.Constr *Awaiting Model Update*>

In [99]:
prob3.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 46 rows, 40 columns and 107 nonzeros
Model fingerprint: 0x801a3e34
Variable types: 20 continuous, 20 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-01, 6e+02]
Found heuristic solution: objective 1147.3456604
Presolve removed 10 rows and 6 columns
Presolve time: 0.00s
Presolved: 36 rows, 34 columns, 87 nonzeros
Found heuristic solution: objective 1146.3456606
Variable types: 17 continuous, 17 integer (0 binary)
Found heuristic solution: objective 1145.3456606

Root relaxation: objective 2.926750e+00, 34 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.926

In [100]:
for v in prob3.getVars():
    print(v.varName, "=", round(v.x))

Devation from fair # Seats for provice 0 = 0
Devation from fair # Seats for provice 1 = 0
Devation from fair # Seats for provice 2 = 1
Devation from fair # Seats for provice 3 = 0
Devation from fair # Seats for provice 4 = 1
Devation from fair # Seats for provice 5 = 0
Devation from fair # Seats for provice 6 = 0
Devation from fair # Seats for provice 7 = 1
Devation from fair # Seats for provice 8 = 1
Devation from fair # Seats for provice 9 = 1
Devation from fair # Seats for provice 10 = 0
Devation from fair # Seats for provice 11 = 0
Devation from fair # Seats for provice 12 = 0
Devation from fair # Seats for provice 13 = 0
Devation from fair # Seats for provice 14 = 0
Devation from fair # Seats for provice 15 = 0
Devation from fair # Seats for provice 16 = 0
Devation from fair # Seats for provice 17 = 0
Devation from fair # Seats for provice 18 = 0
Devation from fair # Seats for provice 19 = 0
Fair # Seats for provice 0 = 22
Fair # Seats for provice 1 = 38
Fair # Seats for provice 2

### 1.4: Opinions.

In [69]:
# Your opinion
# In my opinion, it does not erode the democracy because of the unique structure of the government in the USA. 
# The two mentioned states have the same number of representatives in the US Senate, however, 
# California has 55 electoral votes in the presidential election while South Dakota has only 3. 
# Since democracy is the product of the whole structure (not any single entity) and society, the unfair number of representatives 
# in the US Senate could be compensated by the higher impact of the states in selecting the president. 

# They might have decided to give all the states the same number of representatives in the US Senate to initially keep
# all of them united together. 

## 2: Did You Miss BalancedMilk Co. and Ms. Fraser?

### 2.1: The salesforce problem.

In [387]:
B = [[1, 0.3, -0.1, 0, 0, -0.2, 0, -0.3, 0.4, 0.1],
              [0.2, 1, 0.1, 0.1, -0.1, 0, -0.1, 0.1, 0.3, -0.2],
              [0.2, 0, 1, 0.1, -0.1, 0.2, 0.2, 0, -0.3, 0],
              [-0.1, 0.2, 0, 1, 0.2, 0, 0.2, 0, 0.1, -0.1],
              [0.1, -0.1, -0.1, 0.1, 1, 0.1, 0.1, 0.3, 0, 0.1], 
              [-0.2, -0.2, -0.1, 0.2, 0.2, 1, 0.3, -0.1, 0, -0.2],
              [0.1, 0.2, -0.1, -0.1, 0.2, -0.1, 1, 0.2, 0.1, 0.3],
              [0.1, 0.1, 0.3, -0.1, 0.1, 0.2, 0.2, 1, 0.2, 0.1],
              [0.3, 0.4, -0.2, 0.2, 0.1, 0, 0.1, -0.1, 1, -0.1],
              [-0.1, -0.2, 0.1, 0.1, 0, -0.1, 0.2, 0.2, 0.3, 1]]

# Your solution for 2.1

In [388]:
prob5 = gp.Model("Salesforce")

In [389]:
SalesPeople = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10']
P = len(SalesPeople)

In [390]:
X = prob5.addVars(P, lb = 0, vtype = GRB.CONTINUOUS, name = ["Award for Salesperson_"+str(i+1) for i in range(P)])
Y = prob5.addVars(P, vtype = GRB.BINARY, name = ["Decreased limit for Salesperson_"+str(i+1) for i in range(P)])
Z = prob5.addVars(P, vtype = GRB.BINARY, name = ["Increased limit for Salesperson_"+str(i+1) for i in range(P)])
T = prob5.addVars(P, vtype = GRB.BINARY, name = ["Normal limit for Salesperson_"+str(i+1) for i in range(P)])

In [391]:
prob5.setObjective(sum(B[i][j] * X[j] for j in range(P) for i in range(P))/P, GRB.MAXIMIZE)

In [392]:
# Defining the budget constraint
prob5.addConstr(sum(X[i] for i in range(P)) <= 10000)

<gurobi.Constr *Awaiting Model Update*>

In [393]:
# Defining the minimum payoff constraint
for i in range(P):
    prob5.addConstr(sum(B[i][j]*X[j] for j in range(P)) >= 1500*T[i])

In [394]:
# At most 3 people's payoff can be dropped to 1000.
for i in range(P):
    prob5.addConstr(sum(B[i][j]*X[j] for j in range(P)) >= 1000*Y[i])
    
prob5.addConstr(sum(Y[i] for i in range(P)) <= 3)

<gurobi.Constr *Awaiting Model Update*>

In [395]:
# At least 3 people should receive 2500 or higher payoff.
for i in range(P):
    prob5.addConstr(sum(B[i][j]*X[j] for j in range(P)) >= 2500*Z[i])
    
prob5.addConstr(sum(Z[i] for i in range(P)) >= 3)

<gurobi.Constr *Awaiting Model Update*>

In [396]:
for i in range(P):
    prob5.addConstr(Y[i] + Z[i] + T[i] == 1)

In [397]:
prob5.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 43 rows, 40 columns and 348 nonzeros
Model fingerprint: 0x48f9e83f
Variable types: 10 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e-01, 3e+03]
  Objective range  [9e-02, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve time: 0.00s
Presolved: 43 rows, 40 columns, 348 nonzeros
Variable types: 10 continuous, 30 integer (30 binary)

Root relaxation: objective 2.156417e+03, 33 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2156.41730    0   11          - 2156.41730      -     -    0s
H    0     0                    1910.5778450 2156.41730  12.9%     -    0s
     0     0 2111.70213    0   13 1910.57785 2111.7021

In [398]:
for v in prob5.getVars():
    print(v.varName, "=", round(v.x))

Award for Salesperson_1 = 747
Award for Salesperson_2 = 1491
Award for Salesperson_3 = 27
Award for Salesperson_4 = 1095
Award for Salesperson_5 = 416
Award for Salesperson_6 = 152
Award for Salesperson_7 = 5024
Award for Salesperson_8 = 114
Award for Salesperson_9 = 933
Award for Salesperson_10 = 0
Decreased limit for Salesperson_1 = 0
Decreased limit for Salesperson_2 = 0
Decreased limit for Salesperson_3 = 1
Decreased limit for Salesperson_4 = 0
Decreased limit for Salesperson_5 = 1
Decreased limit for Salesperson_6 = 0
Decreased limit for Salesperson_7 = 0
Decreased limit for Salesperson_8 = 0
Decreased limit for Salesperson_9 = 0
Decreased limit for Salesperson_10 = 1
Increased limit for Salesperson_1 = 0
Increased limit for Salesperson_2 = 0
Increased limit for Salesperson_3 = 0
Increased limit for Salesperson_4 = 1
Increased limit for Salesperson_5 = 0
Increased limit for Salesperson_6 = 0
Increased limit for Salesperson_7 = 1
Increased limit for Salesperson_8 = 0
Increased limi

### 2.2: BalancedMilk in the middle of the crisis.

In [2]:
# importing data
Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7']
Supply = [110, 80, 100, 240, 100, 280, 130]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]

Undersupply_Cost = [15, 15, 15, 15, 15, 15, 15, 15, 15, 15]

In [3]:
prob6 = gp.Model("mid-crisis")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-10


In [4]:
S = len(SupplyCenter)
D = len(DemandMarket)
U = len(Undersupply_Cost)

In [5]:
X = prob6.addVars(S,D, lb = 0, vtype = GRB.CONTINUOUS, name = ["X_"+i+"_"+j for i in SupplyCenter for j in DemandMarket])

In [6]:
prob6.setObjective(sum(Cost[i][j]*X[i,j] for i in range(S) for j in range(D)) + sum((Demand[j] - sum(X[i,j] for i in range(S)))*Undersupply_Cost[j] for j in range(D)), GRB.MINIMIZE)

In [7]:
# defining supply constraints
# the firm can choose not to supply all the produced milk.
for i in range(S):
    prob6.addConstr(sum(X[i,j] for j in range(D)) <= Supply[i])

In [8]:
# defining demand constraints
# the firm can undersupply a demand market but it is not costless; 
# the cost of undersupplying a market has been considered in the revised objective function
for j in range(D):
    prob6.addConstr(sum(X[i,j] for i in range(S)) <= Demand[j])

In [9]:
Y = prob6.addVars(S, vtype=GRB.BINARY, name = ["Additional constraint for supply center "+j for j in SupplyCenter])
Z = prob6.addVars(D, vtype=GRB.BINARY, name = ["Additional constraint for demand market "+j for j in DemandMarket])

In [10]:
prob6.addConstr(sum(X[2,j] for j in range(D)) >= 0.5*Supply[2]*Y[2])
prob6.addConstr(sum(X[3,j] for j in range(D)) >= 0.5*Supply[3]*Y[3])
prob6.addConstr(sum(X[4,j] for j in range(D)) >= 0.5*Supply[4]*Y[4])
prob6.addConstr(Y[2] + Y[3] + Y[4] >= 2)

<gurobi.Constr *Awaiting Model Update*>

In [11]:
prob6.addConstr(sum(X[i,4] for i in range(S)) >= 0.8*Demand[4] - 150*Z[4])
prob6.addConstr(sum(X[i,5] for i in range(S)) >= 0.6*Demand[5] - 150*(1 - Z[4]))
# Since 0.8*D5 = 0.6*D6 = 144, M = 150 is a proper choice (large enough)

<gurobi.Constr *Awaiting Model Update*>

In [15]:
prob6.addConstr(sum(X[i,1] for i in range(S)) >= 0.5*Demand[1] - 90*Z[1])
prob6.addConstr(sum(X[i,7] for i in range(S)) >= 0.5*Demand[7] - 90*(1-Z[1]))
prob6.addConstr(sum(X[i,8] for i in range(S)) >= 0.5*Demand[8] - 90*(1-Z[1]))
# Since D2 = 100, D8 = 90, and D9 = 160, M = 90 is a proper choice (large enough)

<gurobi.Constr *Awaiting Model Update*>

In [16]:
prob6.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 29 rows, 87 columns and 240 nonzeros
Model fingerprint: 0x0f1e5312
Variable types: 70 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 3e+02]

MIP start from previous solve produced solution with objective 15860 (0.01s)
Loaded MIP start from previous solve with objective 15860

Presolve removed 1 rows and 27 columns
Presolve time: 0.00s
Presolved: 28 rows, 60 columns, 202 nonzeros
Variable types: 55 continuous, 5 integer (5 binary)

Root relaxation: objective 1.572500e+04, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 15725.0000    0  

In [17]:
for v in prob6.getVars():
    print(v.varName, "=", round(v.x))

X_S_1_D_1 = 0
X_S_1_D_2 = 0
X_S_1_D_3 = 0
X_S_1_D_4 = 0
X_S_1_D_5 = 0
X_S_1_D_6 = 0
X_S_1_D_7 = 0
X_S_1_D_8 = 0
X_S_1_D_9 = 110
X_S_1_D_10 = 0
X_S_2_D_1 = 0
X_S_2_D_2 = 0
X_S_2_D_3 = 35
X_S_2_D_4 = 0
X_S_2_D_5 = 0
X_S_2_D_6 = 0
X_S_2_D_7 = 0
X_S_2_D_8 = 45
X_S_2_D_9 = 0
X_S_2_D_10 = 0
X_S_3_D_1 = 0
X_S_3_D_2 = 0
X_S_3_D_3 = 0
X_S_3_D_4 = 0
X_S_3_D_5 = 0
X_S_3_D_6 = 100
X_S_3_D_7 = 0
X_S_3_D_8 = 0
X_S_3_D_9 = 0
X_S_3_D_10 = 0
X_S_4_D_1 = 0
X_S_4_D_2 = 100
X_S_4_D_3 = 0
X_S_4_D_4 = 0
X_S_4_D_5 = 0
X_S_4_D_6 = 0
X_S_4_D_7 = 0
X_S_4_D_8 = 0
X_S_4_D_9 = 0
X_S_4_D_10 = 0
X_S_5_D_1 = 90
X_S_5_D_2 = 0
X_S_5_D_3 = 0
X_S_5_D_4 = 0
X_S_5_D_5 = 0
X_S_5_D_6 = 0
X_S_5_D_7 = 0
X_S_5_D_8 = 0
X_S_5_D_9 = 10
X_S_5_D_10 = 0
X_S_6_D_1 = 0
X_S_6_D_2 = 0
X_S_6_D_3 = 0
X_S_6_D_4 = 0
X_S_6_D_5 = 180
X_S_6_D_6 = 0
X_S_6_D_7 = 0
X_S_6_D_8 = 0
X_S_6_D_9 = 0
X_S_6_D_10 = 70
X_S_7_D_1 = 0
X_S_7_D_2 = 0
X_S_7_D_3 = 115
X_S_7_D_4 = 0
X_S_7_D_5 = 0
X_S_7_D_6 = 0
X_S_7_D_7 = 0
X_S_7_D_8 = 0
X_S_7_D_9 = 15
X_S_7_D_10 =

### 2.3 - BalancedMilk before the next crisis.

In [74]:
# importing data
Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51],
        [34, 17, 10, 21, 9, 33, 14, 26, 19, 45]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [110, 80, 100, 240, 100, 280, 130, 440]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]

Undersupply_Cost = [30, 30, 30, 30, 30, 30, 30, 30, 30, 30]
# Your solution for 2.3

In [75]:
prob7 = gp.Model("post-crisis")

In [76]:
S = len(SupplyCenter)
D = len(DemandMarket)
U = len(Undersupply_Cost)

In [77]:
X = prob7.addVars(S,D, lb = 0, vtype = GRB.CONTINUOUS, name = ["X_"+i+"_"+j for i in SupplyCenter for j in DemandMarket])

In [78]:
Y = prob7.addVars(S, vtype = GRB.BINARY, name = ["Supply Center "+str(i+1)+" open" for i in range(S)])

In [79]:
prob7.setObjective(sum(Cost[i][j]*X[i,j] for i in range(S) for j in range(D)) + sum((Demand[j] - sum(X[i,j] for i in range(S)))*Undersupply_Cost[j] for j in range(D)), GRB.MINIMIZE)

In [80]:
# Defining the constraint for limiting the number of shut down supply centers for the best case scenario.
prob7.addConstr(sum(Y[i] for i in range(S)) == 7)

<gurobi.Constr *Awaiting Model Update*>

In [81]:
# defining supply constraints
# The firm is committed to distributing all the milk supplied by the active supply centers.
for i in range(S):
    prob7.addConstr(sum(X[i,j] for j in range(D)) == Supply[i]*Y[i])

In [82]:
# defining demand constraints
# the firm can undersupply a demand market but it is not costless;
# the cost of undersupplying a market has been considered in the revised objective function
for j in range(D):
    prob7.addConstr(sum(X[i,j] for i in range(S)) <= Demand[j])

In [83]:
prob7.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 19 rows, 88 columns and 176 nonzeros
Model fingerprint: 0x00535f13
Variable types: 80 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 2e+02]
Presolve time: 0.01s
Presolved: 19 rows, 88 columns, 176 nonzeros
Variable types: 80 continuous, 8 integer (8 binary)
Found heuristic solution: objective 19990.000000

Root relaxation: objective 1.922583e+04, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    19460.000000 19460.0000  0.00%     -    0s

Explored 1 nodes (23 simplex iterations) in 0.02 seconds (0.00 work units)
Thread c

In [84]:
for v in prob7.getVars():
    print(v.varName, "=", round(v.x))

X_S_1_D_1 = 0
X_S_1_D_2 = 0
X_S_1_D_3 = 0
X_S_1_D_4 = 0
X_S_1_D_5 = 0
X_S_1_D_6 = 0
X_S_1_D_7 = 0
X_S_1_D_8 = 0
X_S_1_D_9 = 110
X_S_1_D_10 = 0
X_S_2_D_1 = 0
X_S_2_D_2 = 0
X_S_2_D_3 = 0
X_S_2_D_4 = 0
X_S_2_D_5 = 0
X_S_2_D_6 = 0
X_S_2_D_7 = 0
X_S_2_D_8 = 0
X_S_2_D_9 = 0
X_S_2_D_10 = 0
X_S_3_D_1 = 0
X_S_3_D_2 = 0
X_S_3_D_3 = 0
X_S_3_D_4 = 0
X_S_3_D_5 = 0
X_S_3_D_6 = 100
X_S_3_D_7 = 0
X_S_3_D_8 = 0
X_S_3_D_9 = 0
X_S_3_D_10 = 0
X_S_4_D_1 = 0
X_S_4_D_2 = 100
X_S_4_D_3 = 0
X_S_4_D_4 = 0
X_S_4_D_5 = 0
X_S_4_D_6 = 140
X_S_4_D_7 = 0
X_S_4_D_8 = 0
X_S_4_D_9 = 0
X_S_4_D_10 = 0
X_S_5_D_1 = 90
X_S_5_D_2 = 0
X_S_5_D_3 = 0
X_S_5_D_4 = 10
X_S_5_D_5 = 0
X_S_5_D_6 = 0
X_S_5_D_7 = 0
X_S_5_D_8 = 0
X_S_5_D_9 = 0
X_S_5_D_10 = 0
X_S_6_D_1 = 0
X_S_6_D_2 = 0
X_S_6_D_3 = 0
X_S_6_D_4 = 0
X_S_6_D_5 = 180
X_S_6_D_6 = 0
X_S_6_D_7 = 0
X_S_6_D_8 = 30
X_S_6_D_9 = 0
X_S_6_D_10 = 70
X_S_7_D_1 = 0
X_S_7_D_2 = 0
X_S_7_D_3 = 80
X_S_7_D_4 = 0
X_S_7_D_5 = 0
X_S_7_D_6 = 0
X_S_7_D_7 = 0
X_S_7_D_8 = 0
X_S_7_D_9 = 50
X_S_7_D_10 =

In [50]:
# Solving the problem for the worst case scenario.
# In comparison with the previous part, the only difference is the sum of Y[i] should be equal to 5 instead of 7

In [63]:
# importing data
Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51],
        [34, 17, 10, 21, 9, 33, 14, 26, 19, 45]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [110, 80, 100, 240, 100, 280, 130, 440]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]

Undersupply_Cost = [30, 30, 30, 30, 30, 30, 30, 30, 30, 30]
# Your solution for 2.3

In [64]:
prob8 = gp.Model("post-crisis")

In [65]:
S = len(SupplyCenter)
D = len(DemandMarket)
U = len(Undersupply_Cost)

In [66]:
X = prob8.addVars(S,D, lb = 0, vtype = GRB.CONTINUOUS, name = ["X_"+i+"_"+j for i in SupplyCenter for j in DemandMarket])

In [67]:
Y = prob8.addVars(S, vtype = GRB.BINARY, name = ["Supply Center "+str(i+1)+" open" for i in range(S)])

In [68]:
prob8.setObjective(sum(Cost[i][j]*X[i,j] for i in range(S) for j in range(D)) + sum((Demand[j] - sum(X[i,j] for i in range(S)))*Undersupply_Cost[j] for j in range(D)), GRB.MINIMIZE)

In [69]:
# Defining the constraint for limiting the number of shut down supply centers for the worst case scenario.
prob8.addConstr(sum(Y[i] for i in range(S)) == 5)

<gurobi.Constr *Awaiting Model Update*>

In [70]:
# defining supply constraints
# The firm is committed to distributing all the milk supplied by the active supply centers.
for i in range(S):
    prob8.addConstr(sum(X[i,j] for j in range(D)) == Supply[i]*Y[i])

In [71]:
# defining demand constraints
# the firm can undersupply a demand market but it is not costless;
# the cost of undersupplying a market has been considered in the revised objective function
for j in range(D):
    prob8.addConstr(sum(X[i,j] for i in range(S)) <= Demand[j])

In [72]:
prob8.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 19 rows, 88 columns and 176 nonzeros
Model fingerprint: 0xb5a61460
Variable types: 80 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 2e+02]
Presolve time: 0.00s
Presolved: 19 rows, 88 columns, 176 nonzeros
Variable types: 80 continuous, 8 integer (8 binary)
Found heuristic solution: objective 23130.000000

Root relaxation: objective 2.249500e+04, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    23020.000000 23020.0000  0.00%     -    0s

Explored 1 nodes (25 simplex iterations) in 0.02 seconds (0.00 work units)
Thread c

In [73]:
for v in prob8.getVars():
    print(v.varName, "=", round(v.x))

X_S_1_D_1 = 0
X_S_1_D_2 = 0
X_S_1_D_3 = 0
X_S_1_D_4 = 0
X_S_1_D_5 = 0
X_S_1_D_6 = 0
X_S_1_D_7 = 0
X_S_1_D_8 = 0
X_S_1_D_9 = 0
X_S_1_D_10 = 0
X_S_2_D_1 = 0
X_S_2_D_2 = 0
X_S_2_D_3 = 0
X_S_2_D_4 = 0
X_S_2_D_5 = 0
X_S_2_D_6 = 0
X_S_2_D_7 = 0
X_S_2_D_8 = 0
X_S_2_D_9 = 0
X_S_2_D_10 = 0
X_S_3_D_1 = 0
X_S_3_D_2 = 0
X_S_3_D_3 = 0
X_S_3_D_4 = 0
X_S_3_D_5 = 0
X_S_3_D_6 = 100
X_S_3_D_7 = 0
X_S_3_D_8 = 0
X_S_3_D_9 = 0
X_S_3_D_10 = 0
X_S_4_D_1 = 0
X_S_4_D_2 = 100
X_S_4_D_3 = 0
X_S_4_D_4 = 0
X_S_4_D_5 = 0
X_S_4_D_6 = 140
X_S_4_D_7 = 0
X_S_4_D_8 = 0
X_S_4_D_9 = 0
X_S_4_D_10 = 0
X_S_5_D_1 = 0
X_S_5_D_2 = 0
X_S_5_D_3 = 0
X_S_5_D_4 = 0
X_S_5_D_5 = 0
X_S_5_D_6 = 0
X_S_5_D_7 = 0
X_S_5_D_8 = 0
X_S_5_D_9 = 0
X_S_5_D_10 = 0
X_S_6_D_1 = 30
X_S_6_D_2 = 0
X_S_6_D_3 = 0
X_S_6_D_4 = 0
X_S_6_D_5 = 180
X_S_6_D_6 = 0
X_S_6_D_7 = 0
X_S_6_D_8 = 0
X_S_6_D_9 = 0
X_S_6_D_10 = 70
X_S_7_D_1 = 0
X_S_7_D_2 = 0
X_S_7_D_3 = 0
X_S_7_D_4 = 0
X_S_7_D_5 = 0
X_S_7_D_6 = 0
X_S_7_D_7 = 0
X_S_7_D_8 = 0
X_S_7_D_9 = 130
X_S_7_D_10 = 0
X